In [1]:
!pip install opencv-python

In [2]:
import cv2
print(cv2.__version__)

4.10.0


In [3]:
!pip install cmake --user


In [4]:
!pip install dlib --no-cache-dir --user


In [5]:
!pip install face_recognition --no-cache-dir --user

In [6]:
import cv2
import numpy as np
import face_recognition as fr

In [7]:
video_capture = cv2.VideoCapture(0)

In [8]:
import os
print(os.getcwd())


C:\Users\MSI


In [9]:
image = fr.load_image_file('Tharukshi.jpg')

In [10]:
image_face_encoding = fr.face_encodings(image)[0] 

In [11]:
known_face_encodings = [image_face_encoding]

In [12]:
known_face_names = ["Tharukshi"]

In [ ]:
while True:
    ret, frame = video_capture.read()
    if not ret:
        print("Failed to capture frame. Exiting.")
        break
    
    # Convert to RGB format
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect face locations
    fc_locations = fr.face_locations(rgb_frame)

    # Proceed only if faces are detected
    if fc_locations:
        try:
            fc_encodings = fr.face_encodings(rgb_frame, fc_locations)
        except Exception as e:
            print(f"Error during face encoding: {e}")
            continue  # Skip this frame if encoding fails

    # Iterate over detected faces (if any)
    for (top, right, bottom, left), face_encoding in zip(fc_locations, fc_encodings):
        matches = fr.compare_faces(known_face_encodings, face_encoding)

        name = "Unknown"
        fc_distances = [np.linalg.norm(known_encoding - face_encoding) for known_encoding in known_face_encodings]

        if len(fc_distances) > 0:  # Ensure distances are available
            match_index = np.argmin(fc_distances)
            if matches[match_index]:
                name = known_face_names[match_index]

        # Draw rectangle and label
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the video feed
    cv2.imshow('Face Detection System', frame)

    # Break on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
video_capture.release()
cv2.destroyAllWindows()
